In [1]:
import math
import random


p = 2**127 - 1      # Mersenne
F = lambda x: x % p

'''
witness: x
output: x^2 = 25

We should proof that we know x as x^2 = 25
'''

# I. Arithmetization
vec_w = [1, x, x^2]
# R1CS build : <A_i, w> * <B_i, w> = <C_i, w>

# Constraint 1: x*x = x^2
# Constraint 1: x^2 = 25

A = [
    [0, 1, 0],
    [0, 0, 1]
]
B = [
    [0, 1, 0],
    [1, 0, 0]
]
C = [
    [0, 0, 1],
    [0, 0, 0]
]

K = [0, 25]


# Constraints -> polynomial 

# Parameters

# Proof gen

# Verification

go
